In [22]:
"""
STAT365 Project: METU Ring Bus Survey Data Preprocessing (v2)
==============================================================
This script performs comprehensive data cleaning and preprocessing
for the Ring Bus survey data, including value standardization.

Key improvements in v2:
- Standardizes inconsistent values in multi-select columns
- Consolidates duplicate meanings (e.g., "Free" and "It's free")
- Creates clean one-hot encoded variables
"""

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('cleaned_metu_ring.csv')
print(f"Original dataset: {df.shape[0]} rows × {df.shape[1]} columns")

Original dataset: 325 rows × 37 columns


In [23]:
# ============================================================
# 1. VALUE STANDARDIZATION MAPPINGS
# ============================================================

# Ring Reasons: Consolidate different phrasings
reasons_mapping = {
    # Free variations
    "It's free": "Free",
    "Free": "Free",

    # No car variations
    "I don't have a personal vehicle": "No private car",
    "No private car": "No private car",

    # Convenient variations
    "Most convenient transportation method": "Most convenient",
    "Most convenient": "Most convenient",

    # Fastest variations
    "Fastest transportation method": "Fastest",
    "Fastest": "Fastest",

    # Environment variations
    "Better for the environment": "Better for environment",
    "Better for environment": "Better for environment",

    # Keep as-is
    "Hard to find parking": "Hard to find parking"
}

# Ring Purposes: Consolidate different phrasings
purposes_mapping = {
    # A1 Gate variations
    "Going to Gate A1": "Going to A1 Gate",
    "To get to A1 Entrance": "Going to A1 Gate",

    # A2 Gate variations
    "Going to Gate A2": "Going to A2 Gate",
    "To get to A2 Entrance": "Going to A2 Gate",

    # Department variations
    "Reaching my department": "Going to department",
    "To get to department": "Going to department",

    # East Dorms variations
    "Going to East Dorms": "Going to East Dorms",
    "To get to East dormitory": "Going to East Dorms",

    # West Dorms variations
    "Going to West Dorms": "Going to West Dorms",
    "To get to West dormitory": "Going to West Dorms",

    # Cafeteria/Dining variations
    "Going to cafeteria": "Going to cafeteria",
    "To get to dining facilities": "Going to cafeteria",

    # Library variations
    "Going to library": "Going to library",
    "To get to the library": "Going to library",

    # Sports variations
    "Going to sports/recreation facilities": "Going to sports facilities",
    "To get to sports/recreation facilities": "Going to sports facilities",

    # Other specific destinations (keep as-is)
    "Going to bus station": "Going to bus station",
    "Going to bazaar/marketplace": "Going to bazaar",
    "Going to A4 exit": "Going to A4 exit",
    "Going to School of Foreign Languages": "Going to School of Foreign Languages"
}

# Route names: Consolidate Navy/Navy Blue
routes_mapping = {
    "Navy (East - East)": "Navy Blue (East - East)",
    "Navy Blue (East - East)": "Navy Blue (East - East)",
    "Brown (A1 - A1)": "Brown (A1 - A1)",
    "Yellow (A2 - East)": "Yellow (A2 - East)",
    "Red (East - A2)": "Red (East - A2)",
    "Purple (A1 - A1)": "Purple (A1 - A1)",
    "Orange (West - A2)": "Orange (West - A2)",
    "Turquoise (West - A2)": "Turquoise (West - A2)"
}

# Time slots: Standardize naming conventions
timeslots_mapping = {
    # Morning - consistent
    "Morning (08:00–11:00)": "Morning (08:00-11:00)",
    "Morning (08:00-11:00)": "Morning (08:00-11:00)",

    # Midday/Noon variations
    "Noon (11:00–15:00)": "Midday (11:00-15:00)",
    "Noon (11:00-15:00)": "Midday (11:00-15:00)",
    "Midday (11:00-15:00)": "Midday (11:00-15:00)",
    "Midday (11:00–15:00)": "Midday (11:00-15:00)",

    # Afternoon/Evening variations
    "Afternoon (15:00–19:00)": "Afternoon (15:00-19:00)",
    "Afternoon (15:00-19:00)": "Afternoon (15:00-19:00)",
    "Evening (15:00-19:00)": "Afternoon (15:00-19:00)",
    "Evening (15:00–19:00)": "Afternoon (15:00-19:00)",

    # Late evening variations
    "Late hours (19:00–23:00)": "Late (19:00-23:00)",
    "Late hours (19:00-23:00)": "Late (19:00-23:00)",
    "Late evening (19:00-23:00)": "Late (19:00-23:00)",
    "Late evening (19:00–23:00)": "Late (19:00-23:00)",
    "Late (19:00–23:00)": "Late (19:00-23:00)",
    "Late (19:00-23:00)": "Late (19:00-23:00)"
}

In [24]:
# ============================================================
# 2. FUNCTION TO STANDARDIZE MULTI-SELECT COLUMNS
# ============================================================

def standardize_multiselect(cell, mapping):
    """
    Standardize values in a comma-separated multi-select cell.

    Parameters:
    -----------
    cell : str or NaN - the cell value containing comma-separated items
    mapping : dict - mapping from original values to standardized values

    Returns:
    --------
    str - standardized comma-separated values (or NaN if input is NaN)
    """
    if pd.isna(cell):
        return cell

    # Split by comma and strip whitespace
    values = [v.strip() for v in str(cell).split(',')]

    # Map each value, keep original if not in mapping
    standardized = []
    for v in values:
        if v in mapping:
            standardized.append(mapping[v])
        elif v.strip():  # Keep non-empty values not in mapping
            standardized.append(v)

    # Remove duplicates while preserving order
    seen = set()
    unique_standardized = []
    for v in standardized:
        if v not in seen:
            seen.add(v)
            unique_standardized.append(v)

    return ', '.join(unique_standardized) if unique_standardized else np.nan

# Apply standardization to multi-select columns
print("\n--- Standardizing multi-select columns ---")

df['Ring_Reasons_Std'] = df['Ring_Reasons'].apply(lambda x: standardize_multiselect(x, reasons_mapping))
print("✓ Ring_Reasons standardized")

df['Ring_Purposes_Std'] = df['Ring_Purposes'].apply(lambda x: standardize_multiselect(x, purposes_mapping))
print("✓ Ring_Purposes standardized")

df['Route_Used_Most_Std'] = df['Route_Used_Most'].apply(lambda x: standardize_multiselect(x, routes_mapping))
print("✓ Route_Used_Most standardized")

df['Weekday_Ring_TimeSlots_Std'] = df['Weekday_Ring_TimeSlots'].apply(lambda x: standardize_multiselect(x, timeslots_mapping))
print("✓ Weekday_Ring_TimeSlots standardized")

df['Weekend_Ring_TimeSlots_Std'] = df['Weekend_Ring_TimeSlots'].apply(lambda x: standardize_multiselect(x, timeslots_mapping))
print("✓ Weekend_Ring_TimeSlots standardized")


--- Standardizing multi-select columns ---
✓ Ring_Reasons standardized
✓ Ring_Purposes standardized
✓ Route_Used_Most standardized
✓ Weekday_Ring_TimeSlots standardized
✓ Weekend_Ring_TimeSlots standardized


In [25]:
# ============================================================
# 3. VERIFY STANDARDIZATION
# ============================================================

def count_unique_values(series):
    """Count unique individual values in a multi-select column"""
    all_values = set()
    for cell in series.dropna():
        values = [v.strip() for v in str(cell).split(',')]
        all_values.update(values)
    return sorted(all_values)

print("\n--- Verification: Unique values after standardization ---")

print(f"\nRing_Reasons_Std ({len(count_unique_values(df['Ring_Reasons_Std']))} unique):")
for v in count_unique_values(df['Ring_Reasons_Std']):
    print(f"  - {v}")

print(f"\nRing_Purposes_Std ({len(count_unique_values(df['Ring_Purposes_Std']))} unique):")
for v in count_unique_values(df['Ring_Purposes_Std']):
    print(f"  - {v}")

print(f"\nRoute_Used_Most_Std ({len(count_unique_values(df['Route_Used_Most_Std']))} unique):")
for v in count_unique_values(df['Route_Used_Most_Std']):
    print(f"  - {v}")

print(f"\nWeekday_Ring_TimeSlots_Std ({len(count_unique_values(df['Weekday_Ring_TimeSlots_Std']))} unique):")
for v in count_unique_values(df['Weekday_Ring_TimeSlots_Std']):
    print(f"  - {v}")

print(f"\nWeekend_Ring_TimeSlots_Std ({len(count_unique_values(df['Weekend_Ring_TimeSlots_Std']))} unique):")
for v in count_unique_values(df['Weekend_Ring_TimeSlots_Std']):
    print(f"  - {v}")


--- Verification: Unique values after standardization ---

Ring_Reasons_Std (6 unique):
  - Better for environment
  - Fastest
  - Free
  - Hard to find parking
  - Most convenient
  - No private car

Ring_Purposes_Std (12 unique):
  - Going to A1 Gate
  - Going to A2 Gate
  - Going to A4 exit
  - Going to East Dorms
  - Going to School of Foreign Languages
  - Going to West Dorms
  - Going to bazaar
  - Going to bus station
  - Going to cafeteria
  - Going to department
  - Going to library
  - Going to sports facilities

Route_Used_Most_Std (7 unique):
  - Brown (A1 - A1)
  - Navy Blue (East - East)
  - Orange (West - A2)
  - Purple (A1 - A1)
  - Red (East - A2)
  - Turquoise (West - A2)
  - Yellow (A2 - East)

Weekday_Ring_TimeSlots_Std (4 unique):
  - Afternoon (15:00-19:00)
  - Late (19:00-23:00)
  - Midday (11:00-15:00)
  - Morning (08:00-11:00)

Weekend_Ring_TimeSlots_Std (4 unique):
  - Afternoon (15:00-19:00)
  - Late (19:00-23:00)
  - Midday (11:00-15:00)
  - Morning (08:00-

In [26]:
# ============================================================
# 4. ORDINAL ENCODING
# ============================================================

print("\n--- Applying ordinal encoding ---")

# Problem frequency mapping (Never=1 to Always=5)
problem_map = {'Never': 1, 'Rarely': 2, 'Sometimes': 3, 'Often': 4, 'Always': 5}

# Transportation frequency mapping
freq_map = {'Never': 1, 'Rarely': 2, 'Sometimes': 3, 'Often': 4, 'Always': 5}

# Weekday Ring frequency mapping
weekday_freq_map = {
    'Never': 0,
    'Very rarely': 1,
    '1–2 times a week': 2,
    '3–4 times a week': 3,
    'Once a day': 4,
    'More than once a day': 5
}

# Weekend Ring frequency mapping
weekend_freq_map = {
    'Never': 0,
    'Rarely (once or twice a month)': 1,
    'Rarely (1–2 times a month)': 1,
    'Sometimes (a few times a month)': 2,
    'Often (every weekend)': 3
}

# Wait time mapping (ordinal)
wait_map = {
    'Less than 5 minutes': 1,
    '5–10 minutes': 2,
    '11–15 minutes': 3,
    '16–20 minutes': 4,
    'More than 20 minutes': 5
}

# Apply problem frequency encoding
problem_cols = [c for c in df.columns if c.startswith('Problem_') and not c.endswith('_Numeric')]
for col in problem_cols:
    df[f'{col}_Numeric'] = df[col].map(problem_map)

# Apply transportation frequency encoding
transport_cols = ['Freq_Taxi', 'Freq_Ring', 'Freq_Walking', 'Freq_Hitchhiking', 'Freq_PersonalVehicle']
for col in transport_cols:
    df[f'{col}_Numeric'] = df[col].map(freq_map)

# Apply weekday/weekend frequency encoding
df['Weekday_Ring_Frequency_Numeric'] = df['Weekday_Ring_Frequency'].map(weekday_freq_map)
df['Weekend_Ring_Frequency_Numeric'] = df['Weekend_Ring_Frequency'].map(weekend_freq_map)

# Apply wait time encoding
df['Avg_Wait_Time_Numeric'] = df['Avg_Wait_Time'].map(wait_map)

print("✓ Ordinal encoding completed")


--- Applying ordinal encoding ---
✓ Ordinal encoding completed


In [27]:

# ============================================================
# 5. ONE-HOT ENCODING FOR STANDARDIZED MULTI-SELECT COLUMNS
# ============================================================

print("\n--- Applying one-hot encoding ---")

def one_hot_encode_multiselect(df, column, prefix, predefined_values=None):
    """
    One-hot encode a comma-separated multi-select column.

    Parameters:
    -----------
    df : DataFrame
    column : str - column name containing comma-separated values
    prefix : str - prefix for new column names
    predefined_values : list - predefined list of values to encode (optional)

    Returns:
    --------
    DataFrame with new binary columns, list of encoded values
    """
    # Get all unique values if not predefined
    if predefined_values is None:
        all_values = set()
        for cell in df[column].dropna():
            values = [v.strip() for v in str(cell).split(',')]
            all_values.update(values)
        predefined_values = sorted(all_values)

    # Create binary columns
    for value in predefined_values:
        # Clean column name: remove special chars, limit length
        clean_name = value.replace(' ', '_').replace('(', '').replace(')', '')
        clean_name = clean_name.replace('-', '_').replace("'", "").replace('/', '_')
        clean_name = clean_name.replace(',', '').replace('.', '')[:25]
        col_name = f"{prefix}_{clean_name}"

        df[col_name] = df[column].apply(
            lambda x: 1 if pd.notna(x) and value in str(x) else 0
        )

    return df, predefined_values

# Define the standardized value lists for one-hot encoding
std_reasons = [
    'Free',
    'No private car',
    'Most convenient',
    'Fastest',
    'Better for environment',
    'Hard to find parking'
]

std_purposes = [
    'Going to A1 Gate',
    'Going to A2 Gate',
    'Going to department',
    'Going to East Dorms',
    'Going to West Dorms',
    'Going to cafeteria',
    'Going to library',
    'Going to sports facilities'
]

std_routes = [
    'Brown (A1 - A1)',
    'Turquoise (West - A2)',
    'Orange (West - A2)',
    'Yellow (A2 - East)',
    'Red (East - A2)',
    'Navy Blue (East - East)',
    'Purple (A1 - A1)'
]

std_timeslots = [
    'Morning (08:00-11:00)',
    'Midday (11:00-15:00)',
    'Afternoon (15:00-19:00)',
    'Late (19:00-23:00)'
]

# Apply one-hot encoding to standardized columns
df, _ = one_hot_encode_multiselect(df, 'Ring_Reasons_Std', 'Reason', std_reasons)
print("✓ Ring_Reasons one-hot encoded (6 columns)")

df, _ = one_hot_encode_multiselect(df, 'Ring_Purposes_Std', 'Purpose', std_purposes)
print("✓ Ring_Purposes one-hot encoded (8 columns)")

df, _ = one_hot_encode_multiselect(df, 'Route_Used_Most_Std', 'Route', std_routes)
print("✓ Route_Used_Most one-hot encoded (7 columns)")

df, _ = one_hot_encode_multiselect(df, 'Weekday_Ring_TimeSlots_Std', 'TimeWkday', std_timeslots)
print("✓ Weekday_Ring_TimeSlots one-hot encoded (4 columns)")

df, _ = one_hot_encode_multiselect(df, 'Weekend_Ring_TimeSlots_Std', 'TimeWkend', std_timeslots)
print("✓ Weekend_Ring_TimeSlots one-hot encoded (4 columns)")


--- Applying one-hot encoding ---
✓ Ring_Reasons one-hot encoded (6 columns)
✓ Ring_Purposes one-hot encoded (8 columns)
✓ Route_Used_Most one-hot encoded (7 columns)
✓ Weekday_Ring_TimeSlots one-hot encoded (4 columns)
✓ Weekend_Ring_TimeSlots one-hot encoded (4 columns)


In [28]:
# ============================================================
# 6. CREATE DERIVED VARIABLES
# ============================================================

print("\n--- Creating derived variables ---")

# Total usage score (weekday + weekend)
df['Total_Usage_Score'] = (
    df['Weekday_Ring_Frequency_Numeric'].fillna(0) +
    df['Weekend_Ring_Frequency_Numeric'].fillna(0)
)

# Problem severity index (average of all problem numeric scores)
problem_numeric_cols = [c for c in df.columns if c.startswith('Problem_') and c.endswith('_Numeric')]
df['Problem_Severity_Index'] = df[problem_numeric_cols].mean(axis=1)

# Heavy user flag (uses Ring bus 3+ times per week on weekdays)
df['Heavy_User'] = (df['Weekday_Ring_Frequency_Numeric'] >= 3).astype(int)

# Simplified residence categories
def simplify_residence(res):
    if pd.isna(res):
        return 'Unknown'
    if 'East' in res:
        return 'East_Dorms'
    elif 'West' in res or 'Isa Demiray' in res:
        return 'West_Dorms'
    elif 'Postgraduate' in res:
        return 'Postgrad_Guesthouse'
    elif 'within walking' in res:
        return 'Off_Campus_Near'
    elif 'far' in res:
        return 'Off_Campus_Far'
    else:
        return 'Other'

df['Residence_Simplified'] = df['Residence'].apply(simplify_residence)

# Route count (how many different routes does each person use)
def count_items(cell):
    if pd.isna(cell):
        return 0
    return len([v.strip() for v in str(cell).split(',') if v.strip()])

df['Route_Count'] = df['Route_Used_Most_Std'].apply(count_items)
df['Purpose_Count'] = df['Ring_Purposes_Std'].apply(count_items)
df['Reason_Count'] = df['Ring_Reasons_Std'].apply(count_items)

# UseMore binary (Yes=1, No=0, Not sure=NaN)
use_more_map = {'Yes': 1, 'No': 0, 'Not sure': np.nan}
df['UseMore_Binary'] = df['UseMore_IfFrequent'].map(use_more_map)

print("✓ Derived variables created")


--- Creating derived variables ---
✓ Derived variables created


In [29]:
# ============================================================
# 7. VERIFY DERIVED VARIABLES
# ============================================================

print("\n--- Verifying derived variables ---")
print(f"  Total_Usage_Score range: {df['Total_Usage_Score'].min():.0f} - {df['Total_Usage_Score'].max():.0f}")
print(f"  Problem_Severity_Index range: {df['Problem_Severity_Index'].min():.2f} - {df['Problem_Severity_Index'].max():.2f}")
print(f"  Heavy_User: {df['Heavy_User'].sum()} users ({df['Heavy_User'].mean()*100:.1f}%)")
print(f"  Route_Count range: {df['Route_Count'].min():.0f} - {df['Route_Count'].max():.0f}")


--- Verifying derived variables ---
  Total_Usage_Score range: 0 - 8
  Problem_Severity_Index range: 1.62 - 4.38
  Heavy_User: 261 users (80.3%)
  Route_Count range: 1 - 7


In [30]:
# ============================================================
# 8. REMOVE REDUNDANT COLUMNS
# ============================================================

print("\n--- Removing redundant columns ---")

# Columns to remove:
cols_to_remove = []

# 1. Standardized columns (intermediate - we have one-hot versions)
std_cols = [c for c in df.columns if c.endswith('_Std')]
cols_to_remove.extend(std_cols)
print(f"✓ Removing {len(std_cols)} standardized (_Std) columns")

# 2. Original multi-select columns (we have one-hot versions)
orig_multiselect = ['Ring_Reasons', 'Ring_Purposes', 'Route_Used_Most',
                    'Weekday_Ring_TimeSlots', 'Weekend_Ring_TimeSlots']
cols_to_remove.extend(orig_multiselect)
print(f"✓ Removing {len(orig_multiselect)} original multi-select columns")

# 3. Original ordinal text columns (we have numeric versions)
orig_ordinal_text = [
    'Freq_Taxi', 'Freq_Ring', 'Freq_Walking', 'Freq_Hitchhiking',
    'Freq_PersonalVehicle', 'Weekday_Ring_Frequency',
    'Weekend_Ring_Frequency', 'Avg_Wait_Time'
]
# Problem text columns (but NOT Problem_Severity_Index which is derived numeric)
problem_text_cols = [c for c in df.columns if c.startswith('Problem_')
                     and not c.endswith('_Numeric')
                     and c != 'Problem_Severity_Index']
orig_ordinal_text.extend(problem_text_cols)
cols_to_remove.extend(orig_ordinal_text)
print(f"✓ Removing {len(orig_ordinal_text)} original ordinal text columns")

# 4. Remove Reason_Count and Purpose_Count from one-hot list (they're derived, keep them)
# But we should remove them from the one-hot column detection later

# Drop the columns
df = df.drop(columns=cols_to_remove, errors='ignore')
print(f"\nTotal removed: {len(cols_to_remove)} columns")
print(f"Remaining: {df.shape[1]} columns")


--- Removing redundant columns ---
✓ Removing 5 standardized (_Std) columns
✓ Removing 5 original multi-select columns
✓ Removing 16 original ordinal text columns

Total removed: 26 columns
Remaining: 69 columns


In [31]:

# ============================================================
# 9. RENAME NUMERIC COLUMNS (remove _Numeric suffix for cleaner names)
# ============================================================

print("\n--- Renaming columns for clarity ---")

rename_map = {}
for col in df.columns:
    if col.endswith('_Numeric'):
        new_name = col.replace('_Numeric', '')
        # Avoid conflicts with existing columns
        if new_name not in df.columns:
            rename_map[col] = new_name

df = df.rename(columns=rename_map)
print(f"✓ Renamed {len(rename_map)} columns (removed _Numeric suffix)")


--- Renaming columns for clarity ---
✓ Renamed 16 columns (removed _Numeric suffix)


In [32]:
# ============================================================
# 10. SAVE PROCESSED DATASET
# ============================================================

output_path = 'metu_ring_processed.csv'
df.to_csv(output_path, index=False)
print(f"\n✓ Processed dataset saved to: {output_path}")



✓ Processed dataset saved to: metu_ring_processed.csv


In [33]:

# ============================================================
# 11. FINAL SUMMARY
# ============================================================

print("\n" + "=" * 70)
print("FINAL DATASET SUMMARY")
print("=" * 70)
print(f"Total rows: {df.shape[0]}")
print(f"Total columns: {df.shape[1]}")

# Recalculate column groups after cleanup
demographic_cols = ['Gender', 'Residence', 'Residence_Simplified', 'Study_Level',
                    'Faculty', 'Department', 'Grad_School', 'Visiting_Faculty_GradSchool',
                    'Affiliation_Program']

satisfaction_cols = ['Weekday_Schedule_Satisfaction', 'Weekend_Schedule_Satisfaction',
                     'Understanding_Routes', 'App_Helping_Ring', 'Overall_Satisfaction']

transport_numeric_cols = ['Freq_Taxi', 'Freq_Ring', 'Freq_Walking',
                          'Freq_Hitchhiking', 'Freq_PersonalVehicle']

usage_numeric_cols = ['Weekday_Ring_Frequency', 'Weekend_Ring_Frequency', 'Avg_Wait_Time']

problem_numeric_cols = [c for c in df.columns if c.startswith('Problem_') and c != 'Problem_Severity_Index']

onehot_reason_cols = [c for c in df.columns if c.startswith('Reason_') and c != 'Reason_Count']
onehot_purpose_cols = [c for c in df.columns if c.startswith('Purpose_') and c != 'Purpose_Count']
onehot_route_cols = [c for c in df.columns if c.startswith('Route_') and c != 'Route_Count']
onehot_timewkday_cols = [c for c in df.columns if c.startswith('TimeWkday_')]
onehot_timewkend_cols = [c for c in df.columns if c.startswith('TimeWkend_')]

derived_cols = ['Total_Usage_Score', 'Problem_Severity_Index', 'Heavy_User',
                'Route_Count', 'Purpose_Count', 'Reason_Count', 'UseMore_Binary']

other_cols = ['UseMore_IfFrequent', 'MostValuable_Improvement', 'Open_Comments']

print("\nColumn groups:")
print(f"  - Demographic: {len([c for c in demographic_cols if c in df.columns])}")
print(f"  - Satisfaction (1-5 scales): {len([c for c in satisfaction_cols if c in df.columns])}")
print(f"  - Transport frequency (numeric): {len([c for c in transport_numeric_cols if c in df.columns])}")
print(f"  - Usage frequency (numeric): {len([c for c in usage_numeric_cols if c in df.columns])}")
print(f"  - Problem frequency (numeric): {len([c for c in problem_numeric_cols if c in df.columns])}")
print(f"  - One-hot Reasons: {len(onehot_reason_cols)}")
print(f"  - One-hot Purposes: {len(onehot_purpose_cols)}")
print(f"  - One-hot Routes: {len(onehot_route_cols)}")
print(f"  - One-hot Time Weekday: {len(onehot_timewkday_cols)}")
print(f"  - One-hot Time Weekend: {len(onehot_timewkend_cols)}")
print(f"  - Derived variables: {len([c for c in derived_cols if c in df.columns])}")
print(f"  - Other: {len([c for c in other_cols if c in df.columns])}")

print("\n" + "=" * 70)
print("FINAL COLUMN LIST")
print("=" * 70)
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

print("\n" + "=" * 70)
print("KEY STATISTICS")
print("=" * 70)
print(f"\nOverall Satisfaction: Mean={df['Overall_Satisfaction'].mean():.2f}, SD={df['Overall_Satisfaction'].std():.2f}")
print(f"Problem Severity Index: Mean={df['Problem_Severity_Index'].mean():.2f}")
print(f"Heavy Users: {df['Heavy_User'].sum()} ({df['Heavy_User'].mean()*100:.1f}%)")
print(f"Would use more if frequent: {(df['UseMore_IfFrequent']=='Yes').sum()} ({(df['UseMore_IfFrequent']=='Yes').mean()*100:.1f}%)")


FINAL DATASET SUMMARY
Total rows: 325
Total columns: 69

Column groups:
  - Demographic: 9
  - Satisfaction (1-5 scales): 5
  - Transport frequency (numeric): 5
  - Usage frequency (numeric): 3
  - Problem frequency (numeric): 8
  - One-hot Reasons: 6
  - One-hot Purposes: 8
  - One-hot Routes: 7
  - One-hot Time Weekday: 4
  - One-hot Time Weekend: 4
  - Derived variables: 7
  - Other: 3

FINAL COLUMN LIST
   1. Gender
   2. Residence
   3. Study_Level
   4. Faculty
   5. Grad_School
   6. Visiting_Faculty_GradSchool
   7. Affiliation_Program
   8. Weekday_Schedule_Satisfaction
   9. Weekend_Schedule_Satisfaction
  10. Understanding_Routes
  11. UseMore_IfFrequent
  12. App_Helping_Ring
  13. MostValuable_Improvement
  14. Overall_Satisfaction
  15. Open_Comments
  16. Department
  17. Problem_LateBus
  18. Problem_TooCrowded
  19. Problem_LongWait
  20. Problem_NotMyRoute
  21. Problem_RouteConfusing
  22. Problem_HoursMismatch
  23. Problem_NotClean
  24. Problem_DriverBehavior
  2